# Convert .reg files to hipscat

## Import libraries

In [1]:
from multiprocessing import cpu_count
from pathlib import Path

import cdshealpix
import dask.distributed
import hipscat
import hipscat.io.file_io
import lsdb
import mom_builder
import numpy as np
import pandas as pd
import pyarrow as pa
from astropy.coordinates import Angle, Latitude, Longitude
from astropy.table import Table
from hipscat.io.paths import pixel_catalog_file
from hipscat.pixel_math.hipscat_id import HIPSCAT_ID_COLUMN, hipscat_id_to_healpix, healpix_to_hipscat_id
from hipscat.pixel_tree.pixel_tree import PixelTree
from hipscat.pixel_tree.pixel_alignment import align_with_mocs
from hipscat.pixel_tree.pixel_alignment_types import PixelAlignmentType
from hipscat_import.catalog.arguments import ImportArguments as HipscatImportArguments
from hipscat_import.pipeline import pipeline_with_client as hipscat_import_pipeline

from mocpy import MOC
from tqdm import tqdm

/ocean/projects/phy210048p/shared/hipscat/raw/hsc/pdr3/elzorro/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Constants

In [2]:
HIGHEST_ORDER = 17  # Smallest tiles to start with
SPLIT_ORDER = 5  # Split building tasks into tiles of this order
# 'hipscat' or 'points_map'
# 'hipscat' uses the whole catalog and reuires COVERAGE_ORDER and POINT_CATALOG_HIPSCAT to be specified
# 'points_map' uses a low-resolution coverage map and requires POINTS_MAP_FITS to be specified
COVERAGE_TYPE = 'hipscat'
COVERAGE_ORDER = 15  # Maximum order to use for coverage map
HIGHEST_HIPSCAT_ORDER = 4  # Maximum order of hipscat partitions
HIPSCAT_PIXEL_THRESHOLD = 10_000_000  # Maximum number of rows per hipscat partition
DEPTH_DELTA = 2  # Specifies the precision of the healpix filtering, higher is better, lower is faster
MOCPY_THREADS = None  # Number of threads to use with mocpy
MOM_BUILDER_THREADS = cpu_count()  # Number of threads to use with mom_builder
DASK_WORKERS = cpu_count()  # Number of dask workers, for hipscat-import
DASK_WORKERS_COVERAGE = None  # Number of dask workers, for coverage map

In [3]:
VALUE_NP_DTYPE = np.dtype(np.uint8)  # dtype object is required by mom_builder
VALUE_PA_TYPE = pa.from_numpy_dtype(VALUE_NP_DTYPE)
VALUE_COLUMN = 'value'  # Name of the value column in the output Hipscat

In [4]:
# REGION_DIR = Path('.')
REGION_DIR = Path('/ocean/projects/phy210048p/shared/hipscat/raw/hsc/pdr3/bright-object-masks')
BOX_REGIONS = REGION_DIR / 'box.reg.I'
DIP_REGIONS = REGION_DIR / 'dip.reg.I.nodups'  # 'dip.reg.I.nodups_head'
STARS_REGIONS = REGION_DIR / 'stars.reg.I.nodups' #  'stars.reg.I.nodups_head'

# For COVERAGE_TYPE == 'hipscat'
POINT_CATALOG_HIPSCAT = Path('/ocean/projects/phy210048p/shared/hipscat/catalogs/hsc/hsc-pdr3-forced-r')
# For COVERAGE_TYPE == 'points_map'
POINTS_MAP_FITS = Path('./hsc-pdr3-forced-r_points_map.fits')  # Coverage map distributed with hipscat

MOM_DIR = Path('./mom_parquet')
HIPSCAT_DIR = Path('./hipscat')
HIPSCAT_NAME = 'hsc_pdr3_mask_i'  # bright star masks for HSC PDR3 I-band

## Read ascii region files and convert to healpix tiles of `HIGHEST_ORDER`

In [5]:
def read_circles_cdshealpix(filename):
    table = Table.read(
        filename,
        format='ascii',
        names=['ra', 'dec', 'radius'],
        units=['deg'] * 3,
    )
    ra = Longitude(table['ra'])
    dec = Latitude(table['dec'])
    radius = Angle(table['radius'])
    
    hp_index = []
    for lon, lat, r in zip(ra, dec, radius):
        idx, _hp_depth, _fully_covered = cdshealpix.cone_search(
            lon,
            lat,
            r,
            depth=HIGHEST_ORDER,
            depth_delta=DEPTH_DELTA,
            flat=True
        )
        hp_index.extend(idx)
        
    return np.array(hp_index, dtype=np.uint64)


def read_circles_mocpy(filename):
    table = Table.read(
        filename,
        format='ascii',
        names=['ra', 'dec', 'radius'],
        units=['deg'] * 3,
    )
    ra = Longitude(table['ra'])
    dec = Latitude(table['dec'])
    radius = Angle(table['radius'])
    
    mocs = MOC.from_cones(
        lon=ra,
        lat=dec,
        radius=radius,
        max_depth=HIGHEST_ORDER,
        delta_depth=DEPTH_DELTA,
        n_threads=MOCPY_THREADS,
    )
    hp_index = np.concatenate([moc.flatten() for moc in mocs])
    return hp_index


# mocpy is faster, even for a single thread
read_circles = read_circles_mocpy


# We use mocpy here because cdshealpix doesn't support box searches
def read_boxes(filename):
    table = Table.read(
        filename,
        format='ascii',
        names=['ra_c', 'dec_c', 'width', 'height'],
        units=['deg'] * 4
    )
    ra_center = Longitude(table['ra_c'])
    dec_center = Latitude(table['dec_c'])
    width = Angle(table['width'])
    height = Angle(table['height'])
    
    width_larger = width > height
    a = np.where(width_larger, 0.5 * width, 0.5 * height)
    b = np.where(width_larger, 0.5 * height, 0.5 * width)
    angle = np.where(width_larger, Angle(90, 'deg'), 0)
    
    mocs = MOC.from_boxes(
        lon=ra_center,
        lat=dec_center,
        a=a,
        b=b,
        angle=angle,
        max_depth=HIGHEST_ORDER,
        n_threads=MOCPY_THREADS,
    )
    hp_index = np.concatenate([moc.flatten() for moc in mocs])
    return hp_index

In [6]:
%time dip_index = read_circles(DIP_REGIONS)
%time stars_index = read_circles(STARS_REGIONS)
%time box_index = read_boxes(BOX_REGIONS)

CPU times: user 5min 41s, sys: 3min 26s, total: 9min 7s
Wall time: 1min 22s
CPU times: user 2min 47s, sys: 183 ms, total: 2min 47s
Wall time: 7.96 s


### Group masks by pixels of `SPLIT_ORDER`

In [7]:
def parent(index, child_order, parent_order):
    delta_order = np.array(child_order - parent_order, dtype=np.uint64)
    assert np.all(delta_order >= 0), 'parent_order must not be lower than the child_order'
    # For some numpyish reason, we cannot just do >>
    shift_bits = np.array(2, dtype=np.uint64) * delta_order
    return np.right_shift(np.array(index, dtype=np.uint64), shift_bits)


def first_child(index, parent_order, child_order):
    delta_order = np.array(child_order - parent_order, dtype=np.uint64)
    assert np.all(delta_order >= 0), 'parent_order must not be lower than the child_order'
    # For some numpyish reason, we cannot just do <<
    shift_bits = np.array(2, dtype=np.uint64) * delta_order
    return np.left_shift(np.array(index, dtype=np.uint64), shift_bits)


def children(index, parent_order, child_order):
    delta_order = child_order - parent_order
    assert delta_order >= 0, 'parent_order must not be lower than the child_order'
    first = first_child(index, parent_order, child_order)
    n_children = 4 ** delta_order
    children = first.reshape(-1, 1) + np.arange(n_children, dtype=np.uint64)
    return children.reshape(-1)


def group_by_order(hp_index, hp_order, split_order):
    # Sort and get unique indexes
    hp_index = np.unique(hp_index)
    
    split_order_npix = 12 * (4 ** split_order)
    split_order_indexes = np.arange(1, split_order_npix, dtype=np.uint64)
    hp_order_separation_indexes = first_child(split_order_indexes, split_order, hp_order)
    
    # Split input indexes by the location in split_order tiles
    split_tile_boundaries = np.searchsorted(hp_index, hp_order_separation_indexes)
    lookup = np.split(hp_index, split_tile_boundaries)
    
    return lookup

In [8]:
%%time

dip_lookup = group_by_order(dip_index, HIGHEST_ORDER, SPLIT_ORDER)
del dip_index
stars_lookup = group_by_order(stars_index, HIGHEST_ORDER, SPLIT_ORDER)
del stars_index
box_lookup = group_by_order(box_index, HIGHEST_ORDER, SPLIT_ORDER)
del box_index

CPU times: user 5min 13s, sys: 13.8 s, total: 5min 27s
Wall time: 5min 28s


### Prepare coverage map

Type of the source data is specified by `COVERAGE_TYPE`:
- `COVERAGE_TYPE = 'points_map'` uses `points_map.fits` built during hipscat-import process, it is a low-resolution but fast method
- `COVERAGE_TYPE = 'hipscat'` uses the whole point-source hipscat catalog to built a coverage map with specified `COVERAGE_ORDER`

In [9]:
%%time

def process_points_map():
    coverage = hipscat.io.file_io.read_fits_image(POINTS_MAP_FITS)
    # Convert coverage to bool and then convert to the target type
    coverage = np.asarray(coverage > 0, dtype=VALUE_NP_DTYPE)

    # next I bascially copy HealpixDataset._read_moc_from_point_map()
    # https://github.com/astronomy-commons/hipscat/blob/04596506011ec5f021ebe612dcf7f5e8e9044795/src/hipscat/catalog/healpix_dataset/healpix_dataset.py#L114
    
    # length of the file is 12 * 4**order
    coverage_order_float = 0.5 * np.log2(coverage.shape[0] / 12)
    coverage_order = int(coverage_order_float)
    assert coverage_order == coverage_order_float, 'coverage_order must be an integer'
    assert coverage_order <= HIGHEST_ORDER, 'coverage_order is larger than HIGHEST_ORDER, cast coverage map to HIGHEST_ORDER'
    
    return coverage, coverage_order


class Coverage:
    def __init__(self, hipscat_catalog, split_order):
        self.hipscat_catalog = hipscat_catalog
        self.split_order = split_order
        
        self.pixel_mapping = self._pixel_mapping(hipscat_catalog, split_order)
        
        # import os
        # from itertools import chain
        # for path in chain.from_iterable(self.pixel_mapping):
        #     if path is None:
        #         continue
        #     assert os.path.exists(path)
    
    @staticmethod
    def _pixel_mapping(hipscat_catalog, split_order):
        split_order_tiles = np.arange(12 * 4**split_order)
        split_order_ranges = np.stack(
            [split_order_tiles, split_order_tiles + 1],
            axis=-1
        )
        split_order_tree = PixelTree(split_order_ranges, split_order)
        pixel_alignment = align_with_mocs(
            left_tree=split_order_tree,
            right_tree=hipscat_catalog.pixel_tree,
            left_moc=None,
            right_moc=hipscat_catalog.moc,
            alignment_type=PixelAlignmentType.LEFT,
        )
        mapping_df = pixel_alignment.pixel_mapping.copy()
        mapping_df['join_path'] = mapping_df.apply(
            lambda row: pixel_catalog_file(
                hipscat_catalog.catalog_base_dir,
                row['join_Norder'],
                row['join_Npix']
            ) if row['join_Norder'] is not None else None,
            axis=1,
        )
        # Series with index being [0..12 * 4**split_order)
        # and values to be a list of paritition paths in the catalog.
        # Each patition intersects with a given tile of split_order
        return mapping_df.groupby('primary_Npix')['join_path'].agg(
            lambda values: list(x for x in values if x is not None),
        )
    
    def get_split_tile(self, split_index, target_order):
        hipscat_id_min = healpix_to_hipscat_id(self.split_order, split_index)
        hipscat_id_max = healpix_to_hipscat_id(self.split_order, split_index + 1)
        
        paths = self.pixel_mapping[split_index]
        if len(paths) == 0:
            return np.array([], dtype=np.uint64)
        
        indexes = []
        for path in paths:
            df = hipscat.io.file_io.file_io.read_parquet_file_to_pandas(
                path,
                storage_options=self.hipscat_catalog.storage_options,
                # Load index only
                columns=[],
            )
            # filter pixels to be within given split tile
            # .loc[] does include end of the interval
            df = df.loc[hipscat_id_min:hipscat_id_max]
            
            hipscat_id = df.index.to_numpy()
            target_index = np.unique(hipscat_id_to_healpix(hipscat_id, target_order))
            
            indexes.append(target_index)

        return np.concatenate(indexes, dtype=np.uint64)


if COVERAGE_TYPE == 'points_map':
    coverage, coverage_order = process_points_map()
    
    def request_coverage(split_index):
        indexes_coverage_order = parent(first_index, HIGHEST_ORDER, coverage_order)
        return np.nonzero(coverage[indexes_coverage_order])[0]
elif COVERAGE_TYPE == 'hipscat':
    assert COVERAGE_ORDER <= HIGHEST_ORDER, 'Coverage order must not be larger than highest order'
    
    hipscat_catalog = hipscat.read_from_hipscat(POINT_CATALOG_HIPSCAT)
    coverage = Coverage(hipscat_catalog, SPLIT_ORDER)
    
    def request_coverage(split_index):
        coverage_order_index = coverage.get_split_tile(split_index, COVERAGE_ORDER)
        # Possible edge-case optimization
        # if COVERAGE_ORDER == HIGHEST_ORDER:
        #     return coverage_order_index
        return children(coverage_order_index, COVERAGE_ORDER, HIGHEST_ORDER)
else:
    raise ValueError(f'Unknown COVERAGE_TYPE = "{COVERAGE_TYPE}"')

CPU times: user 217 ms, sys: 262 ms, total: 479 ms
Wall time: 2.16 s


## Build Multi-Order Continious Map (MOM)

### Define a function to get `HIGHEST_ORDER` tile values

In [10]:
def get_value(order, indexes):
    assert order == HIGHEST_ORDER
    
    first_index = indexes[0]
    top_index = parent(first_index, HIGHEST_ORDER, SPLIT_ORDER)

    # Get HIGHEST_ORDER indexes and shift them to start from 0
    cover = request_coverage(top_index) - first_index
    dip = dip_lookup[top_index] - first_index
    stars = stars_lookup[top_index] - first_index
    box = box_lookup[top_index] - first_index
    
    values = np.zeros(indexes.shape, dtype=VALUE_NP_DTYPE)
    
    # coverage is the first bit, coverage array is either 0 or 1
    values[cover] |= 1
    # Masks arrays can be safely shifted by the smallest index value
    # dip is the second bit
    values[dip] |= 1 << 2
    # stars and their boxes is the third bit
    values[stars] |= 1 << 3
    values[box] |= 1 << 3
    
    return values

### Define a MOM merging strategy

In [11]:
mom_merger = mom_builder.MOMMerger(
    # Each healpix tree leaf would have a single value
    # It is a coincidence that we have a column named 'value' 
    state='value',
    # If four children have the same value, they are merged into a single parent
    merger='equal',
    dtype=VALUE_NP_DTYPE,
)

### Initialize parquet writer object

In [12]:
from writer import Writer

writer = Writer(
    MOM_DIR,
    col_name=VALUE_COLUMN,
    col_type=VALUE_PA_TYPE,
)

### Run MOM building pipeline

In [13]:
%%time

generator = mom_builder.gen_mom_from_fn(
    fn=get_value,
    max_norder=HIGHEST_ORDER,
    split_norder=SPLIT_ORDER,
    merger=mom_merger,
    n_threads=MOM_BUILDER_THREADS,
    tiles_consumer=writer.write,  # immediately write tiles to parquet, output nothing from the generator
)

# Generator yields nothing, because tiles_consumer returns nothing
for _ in tqdm(generator, total=12 * 4**SPLIT_ORDER + 1):
    pass

del dip_lookup, stars_lookup, box_lookup, coverage

100%|██████████| 12289/12289 [03:48<00:00, 53.86it/s] 

CPU times: user 1h 55min 13s, sys: 4min 24s, total: 1h 59min 38s
Wall time: 3min 48s


## Create Hipscat with `hipscat-import`

In [16]:
%%time

hipscat_import_args = HipscatImportArguments(
    input_path=MOM_DIR,
    file_reader='parquet',
    output_path=HIPSCAT_DIR,
    output_artifact_name=HIPSCAT_NAME,
    # We have _hipscat_index predefined, it points to the "begging" of a tile
    # (smallest index within all order-19 children)
    use_hipscat_index=True,
    add_hipscat_index=False,
    highest_healpix_order=HIGHEST_HIPSCAT_ORDER,
    pixel_threshold=HIPSCAT_PIXEL_THRESHOLD,
    # Turn off optimization needed for point catalogs
    drop_empty_siblings=False,
)

with dask.distributed.Client(n_workers=DASK_WORKERS) as client:
    display(client)
    hipscat_import_pipeline(hipscat_import_args, client)

Planning  : 100%|██████████| 5/5 [00:00<00:00, 248.17it/s]


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 128
Total threads: 128,Total memory: 247.07 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42821,Workers: 128
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 247.07 GiB
Comm: tcp://127.0.0.1:38905,Total threads: 1
Dashboard: http://127.0.0.1:33247/status,Memory: 1.93 GiB
Nanny: tcp://127.0.0.1:38181,


Finishing : 100%|██████████| 5/5 [00:02<00:00,  1.85it/s]
2024-08-21 13:36:24,709 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/ocean/projects/phy210048p/shared/hipscat/raw/hsc/pdr3/elzorro/venv/lib/python3.11/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/ocean/projects/phy210048p/shared/hipscat/raw/hsc/pdr3/elzorro/venv/lib/python3.11/site-packages/distributed/worker.py", line 1250, in heartbeat
    response = await retry_operation(
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/ocean/projects/phy210048p/shared/hipscat/raw/hsc/pdr3/elzorro/venv/lib/python3.11/site-packages/distributed/utils_comm.

CPU times: user 1min 21s, sys: 35.8 s, total: 1min 57s
Wall time: 2min 3s


### Load catalog with LSDB

In [17]:
catalog = lsdb.read_hipscat(HIPSCAT_DIR / HIPSCAT_NAME)
with dask.distributed.Client(n_workers=DASK_WORKERS) as client:
    display(client)
    df = catalog.query("value == 13").head(100)
display(df)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 128
Total threads: 128,Total memory: 247.07 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38689,Workers: 128
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 247.07 GiB
Comm: tcp://127.0.0.1:35265,Total threads: 1
Dashboard: http://127.0.0.1:40561/status,Memory: 1.93 GiB
Nanny: tcp://127.0.0.1:41347,


2024-08-21 13:36:47,779 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/ocean/projects/phy210048p/shared/hipscat/raw/hsc/pdr3/elzorro/venv/lib/python3.11/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/ocean/projects/phy210048p/shared/hipscat/raw/hsc/pdr3/elzorro/venv/lib/python3.11/site-packages/distributed/worker.py", line 1250, in heartbeat
    response = await retry_operation(
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/ocean/projects/phy210048p/shared/hipscat/raw/hsc/pdr3/elzorro/venv/lib/python3.11/site-packages/distributed/utils_comm.py", line 459, in retry_operation
    return await retry(


,pixel_Norder,pixel_Npix,value,Norder,Dir,Npix
_hipscat_index,,,,,,
12000895814336512,17,178827283,13,3,0,0
12000896015663104,17,178827286,13,3,0,0
12000896082771968,17,178827287,13,3,0,0
12000896149880832,16,44706822,13,3,0,0
12000896418316288,16,44706823,13,3,0,0
...,...,...,...,...,...,...
36037785043337216,16,134251211,13,3,0,2
36037786385514496,15,33562804,13,3,0,2
36037842220089344,16,134251424,13,3,0,2
